# Temperature Variance and Continental Geometry

Analysis scripts for Neumann and Lutsko (2022)

## Winter climatologies

Start with winter climate of control run, then plots pairs of simulations

In [ ]:
def processdata(data):
    #Get variables for making climatology plots
    
    #Start by loading data:
    data=getdata(opt)
    
    #Now have to be carefyl not to overload memory:
    t_850 = data['temp'].interp(pfull=850)
    yr = len(t_850[:, 0, 0]) / 365
    yr = int(yr) - 1
    
    #get winter temperatures and variance
    wint_temp_850 = np.zeros( ( (( yr, 90, 128, 256))))

    a = 675
    for i in range( yr ):
        wint_temp_850[i, :, :, :] = t_850[a: a + 90, :, :]
        print(i)
        a += 360
    temp_850 = np.mean( np.mean(wint_temp_850, axis = 0), axis = 0)
    var_temp = np.mean( np.var(wint_temp_850, axis = 1), axis = 0)

    #clean-up
    t_850 = []
    wint_temp_850 = []
    
    #Winds:
    u_850 = data['ucomp'].interp(pfull=850)
    v_850 = data['vcomp'].interp(pfull=850)

    wint_u_850 = np.zeros( ( (( yr, 90, 128, 256))))
    wint_v_850 = np.zeros( ( (( yr, 90, 128, 256))))

    a = 675
    for i in range( yr):
        wint_u_850[i, :, :, :] = u_850[a: a + 90, :, :]
        wint_v_850[i, :, :, :] = v_850[a: a + 90, :, :]
        print(i)
        a += 360
    u_850 = np.mean( np.mean(wint_u_850, axis = 0), axis = 0)
    v_850 = np.mean( np.mean(wint_v_850, axis = 0), axis = 0)

    #clean-up
    wint_u_850 = []
    wint_v_850 = []

    #surface pressure
    ps_850 = data['ps']
    wint_press_850 = np.zeros( ( (( yr, 90, 128, 256))))
    a = 675
    for i in range( yr ):
        wint_press_850[i, :, :, :] = ps_850[a: a+90, :, :]
        print(i)
        a += 360
    mean_press = np.mean( np.mean(wint_press_850, axis = 0), axis = 0)

    ps_850 = []
    wint_press_850 = []
    
    #Now get gradients
    gradY, gradX = gradient(temp_850, np.array(data.lat[:]), np.array(data.lon[:]))
    #square:
    gradY = gradY ** 2
    gradX = gradX ** 2
    
    #zonal anomalies:
    Zsub_temp = temp_850 - np.mean(temp_850, axis = 1)[:, np.newaxis]
    Zsub_press = mean_press - np.mean(mean_press, axis = 1)[:, np.newaxis]
    Zsub_ucomp = u_850 - np.mean(u_850, axis = 1)[:, np.newaxis]
    Zsub_vcomp = v_850 - np.mean(v_850, axis = 1)[:, np.newaxis]
    
    lon = np.array(data.lon[:])
    lat = np.array(data.lat[:])
    
    return var_temp, Zsub_temp, gradY, gradX, Zsub_press, u_850, v_850, Zsub_ucomp, Zsub_vcomp, lon, lat

def gradient(mean_temp, lat, lon):
    gradY = np.zeros((128,256))
    gradX = np.zeros((128,256))
    y=len(lat)
    x=len(lon)
    
    rad_lat = lat * np.pi / 180.
    rad_lon = lon * np.pi / 180.
    
    cos_mean_temp = np.zeros((128,256))

    for i in range( len(lat)):
        cos_mean_temp[i, :]=mean_temp[i, :]
        if i==0:
            gradY[i, :] = ((mean_temp[ 1, :] - mean_temp[ 0, :]) / (rad_lat[ 1] - rad_lat[ 0])) / 6.371
        elif i== y-1:
            gradY[i, :] = ((mean_temp[i - 1, :] - mean_temp[ i - 2, :]) / (rad_lat[i - 1] - rad_lat[i - 2])) / 6.371
        else:
            gradY[i, :] = ((mean_temp[i + 1, :] - mean_temp[ i - 1, :]) / (rad_lat[i + 1] - rad_lat[i - 1])) / 6.371
    
    cos_mean_temp = mean_temp /np.cos(rad_lat[:, np.newaxis])
    for i in range( len(lon)):        
        if i==0:
            gradX[:, i] = ((cos_mean_temp[ :, 1] - cos_mean_temp[ :, 0]) / (rad_lon[ 1] - rad_lon[ 0])) / 6.371
        elif i== x-1:
            gradX[:, i] = ((cos_mean_temp[:, i-1] - cos_mean_temp[ :, i-2])/ (rad_lon[i - 1] - rad_lon[i - 2])) / 6.371
        else:
            gradX[:, i] = ((cos_mean_temp[:, i+1] - cos_mean_temp[ :, i-1])/ (rad_lon[i + 1] - rad_lon[i - 1])) / 6.371
    return gradY, gradX

def getdata(x1,x2,x3,x4):
    return xr.open_mfdataset(('/home/nneumann/trial_runs/'+x1+'/atmos_daily.nc','/home/nneumann/trial_runs/'+x2+'/atmos_daily.nc','/home/nneumann/trial_runs/'+x3+'/atmos_daily.nc','/home/nneumann/trial_runs/'+x4+'/atmos_daily.nc'))

#For plotting land outlines:
def getland(x):
    datal=xr.open_dataset("/home/nneumann/input/"+x+".nc")
    land=(datal["land_mask"][:,:])
    return land

In [ ]:
#Store maximum gradients and variances. Have extra values for two continents
max_vars = np.zeros( 11 )
max_y = np.zeros( 11 )
max_x = np.zeros( 11 )
max_sum = np.zeros( 11 )


In [ ]:
#Process control run data
var_temp, Zsub_temp, gradY, gradX, Zsub_press, u_850, v_850, Zsub_ucomp, Zsub_vcomp, lon, lat  = processdata( "control" )

max_vars[0] = np.max(temp_var[90:]) #only look in NH
max_y[0] = np.max(gradY[90:] ** 2)
max_x[0] = np.max(gradX[90:] ** 2)
max_sum[0] = np.max(gradY[90:] ** 2 + gradX[90:] ** 2)

In [ ]:
#Control run
fig = plt.figure(figsize=(12,8))

land = getland("control")
ax1 = plt.subplot(3, 2, 1)
plt.title("a) Temperature Variance",size=16, loc = "left")

levs = np.arange(6., 99, 3.)
plt.contourf( lon, lat, var_temp, levs, cmap = plt.cm.Reds)
cs = plt.colorbar(ticks = [10., 30., 50., 70., 90.])
cs.set_label("[K$^2$]", fontsize = 14)
cs.ax.tick_params(labelsize=12)

plt.contour(lon, lat, land, levels=1,colors='k', linewidths = 0.5)
plt.ylim([0., 90.])

plt.yticks([0., 30., 60., 90.], fontsize = 12)
plt.xticks(fontsize = 0)
plt.ylabel("Latitude", fontsize = 14)

Q = plt.quiver(lon[::8],lat[::4], u_850[::4,::8], v_850[::4,::8], scale = 120)
ax1.quiverkey(Q, X=1.08, Y=1.08, U=5,
             label='5ms$^{-1}$', labelpos='E')


ax1 = plt.subplot(3, 2, 2)
plt.title("b) Temperature Anomalies",size=16, loc = "left")

levs = np.arange(-8., 8.3, 0.3)
plt.contourf( lon, lat, Zsub_temp, levs, cmap = plt.cm.coolwarm)
cs = plt.colorbar(ticks = [-6., -3., 0., 3., 6.])
cs.set_label("[K]", fontsize = 14)
cs.ax.tick_params(labelsize=12)

plt.contour(lon, lat, land, levels=1,colors='k', linewidths = 0.5)
plt.ylim([0., 90.])

plt.yticks(fontsize = 0)
plt.xticks(fontsize = 0)

ax1 = plt.subplot(3, 2, 3)
plt.title("c) ${\\left(\\frac{\\partial \\overline{T}}{\\partial x}\\right)}^2$",size=16, loc = "left")

levs = np.arange(0.2, 1.6, 0.1)
plt.contourf( lon, lat, gradX / 100., levs, cmap = plt.cm.Reds)
cs = plt.colorbar(ticks = [0., 0.5, 1., 1.5])
cs.set_label("[(K/100km)$^2$]", fontsize = 14)
cs.ax.tick_params(labelsize=12)

plt.contour(lon, lat, land, levels=1,colors='k', linewidths = 0.5)
plt.ylim([0., 90.])

plt.yticks([0., 30., 60., 90.], fontsize = 12)
plt.xticks(fontsize = 0)
plt.ylabel("Latitude", fontsize = 13)

ax1 = plt.subplot(3, 2, 4)
plt.title("d) ${\\left(\\frac{\\partial \\overline{T}}{\\partial y}\\right)}^2$",size=16, loc = "left")

plt.contourf( lon, lat, gradY / 100., levs, cmap = plt.cm.Reds)
cs = plt.colorbar(ticks = [0., 0.5, 1., 1.5])
cs.set_label("[(K/100km)$^2$]", fontsize = 13)
cs.ax.tick_params(labelsize=12)

plt.contour(lon, lat, land, levels=1,colors='k', linewidths = 0.5)
plt.ylim([0., 90.])

plt.yticks(fontsize = 0)
plt.xticks([0., 60., 120., 180., 240., 300., 360.],fontsize = 12)
plt.xlabel("Longitude", fontsize = 14)


ax1 = plt.subplot(3, 2, 5)
plt.title("e) Surface Pressure Anomalies",size=16, loc = "left")

levs = np.arange(-12., 12.5, 0.5)
plt.contourf( lon, lat, Zsub_press / 100., levs, cmap = plt.cm.coolwarm)
cs = plt.colorbar(ticks = [-10., -5., 0., 5., 10.])
cs.set_label("[hPa]", fontsize = 14)
cs.ax.tick_params(labelsize=12)
plt.contour(lon, lat, land, levels=1,colors='k', linewidths = 0.5)
Q = plt.quiver(lon[::8],lat[::4], Zsub_ucomp[::4,::8], Zsub_vcomp[::4,::8], scale = 60)
ax1.quiverkey(Q, X=1.08, Y=1.08, U=2,
             label='2ms$^{-1}$', labelpos='E')

plt.ylim([0., 90.])

plt.yticks([0., 30., 60., 90.], fontsize = 12)
plt.xticks([0., 60., 120., 180., 240., 300., 360.],fontsize = 12)

plt.xlabel("Longitude", fontsize = 14)
plt.ylabel("Latitude", fontsize = 14)

fig.tight_layout()

plt.savefig("Control_land.png")
plt.savefig("Control_land.pdf")

Now do pairs of simulations:

In [ ]:
#Narrow and long
var_temp, Zsub_temp, gradY, gradX, Zsub_press, u_850, v_850, Zsub_ucomp, Zsub_vcomp, lon, lat  = get_data( "narrow")
var_temp2, Zsub_temp2, gradY2, gradX2, Zsub_press2, u_8502, v_8502, Zsub_ucomp2, Zsub_vcomp2, lon, lat  = get_data( "long")

max_vars[1] = np.max(temp_var[90:]) #only look in NH
max_y[1] = np.max(gradY[90:] ** 2)
max_x[1] = np.max(gradX[90:] ** 2)
max_sum[1] = np.max(gradY[90:] ** 2 + gradX[90:] ** 2)

max_vars[2] = np.max(temp_var2[90:]) #only look in NH
max_y[2] = np.max(gradY2[90:] ** 2)
max_x[2] = np.max(gradX2[90:] ** 2)
max_sum[2] = np.max(gradY2[90:] ** 2 + gradX2[90:] ** 2)

In [ ]:
fig = plt.figure(figsize=(12,16))

labels = ["a) Narrow", "c)", "e)", "g)", "i)"]
labels2 = ["b) Long", "d)", "f)", "h)", "j)"]

opt = ["narrow", "long"]
for i in range( 2 ):
    
    land = getland(opt[i])
    
    ax1 = plt.subplot(5, 2, 1 + i)
    if i == 0:
        plt.title(labels[0],size=16, loc = "left")
    else:
        plt.title(labels2[0],size=16, loc = "left")

    levs = np.arange(6., 99, 3.)
    if i == 0:
        C1 = plt.contourf( lon, lat, var_temp, levs, cmap = plt.cm.Reds)
    else:
        C1 = plt.contourf( lon, lat, var_temp2, levs, cmap = plt.cm.Reds)
        
    plt.contour(lon, lat, land, levels=1,colors='k', linewidths = 0.5)
    plt.ylim([0., 90.])

    if i == 0:
        plt.yticks([0., 30., 60., 90.], fontsize = 14)
        plt.ylabel("Latitude", fontsize = 16)
    else:
        plt.yticks([0., 30., 60., 90.], fontsize = 0)

    plt.xticks(fontsize = 0)


    if i == 0:
        Q = plt.quiver(lon[::8],lat[::4], u_850[::4,::8], v_850[::4,::8], scale = 120)
    else:
        Q = plt.quiver(lon[::8],lat[::4], u_8502[::4,::8], v_8502[::4,::8], scale = 120)

    ax1.quiverkey(Q, X=0.86, Y=1.05, U=5, label='5ms$^{-1}$', labelpos='E', fontproperties={'size': '12'})


    ax1 = plt.subplot(5, 2, 3 + i)
    if i == 0:
        plt.title(labels[1],size=16, loc = "left")
    else:
        plt.title(labels2[1],size=16, loc = "left")

    levs = np.arange(-8., 8.3, 0.3)
    if i == 0:
        C2 = plt.contourf( lon, lat, Zsub_temp, levs, cmap = plt.cm.coolwarm)
    else:
        C2 = plt.contourf( lon, lat, Zsub_temp2, levs, cmap = plt.cm.coolwarm)
        
    plt.contour(lon, lat, land, levels=1,colors='k', linewidths = 0.5)
    plt.ylim([0., 90.])

    if i == 0:
        plt.yticks([0., 30., 60., 90.], fontsize = 14)
        plt.ylabel("Latitude", fontsize = 16)
    else:
        plt.yticks([0., 30., 60., 90.], fontsize = 0)

    plt.xticks(fontsize = 0)

    ax1 = plt.subplot(5, 2, 5 + i)
    if i == 0:
        plt.title(labels[2],size=16, loc = "left")
    else:
        plt.title(labels2[2],size=16, loc = "left")

    levs = np.arange(0.2, 1.6, 0.1)
    if i == 0:
        C3 = plt.contourf( lon, lat, gradX / 100., levs, cmap = plt.cm.Reds)
    else:
        C3 = plt.contourf( lon, lat, gradX2 / 100., levs, cmap = plt.cm.Reds)
        
    plt.contour(lon, lat, land, levels=1,colors='k', linewidths = 0.5)
    plt.ylim([0., 90.])

    if i == 0:
        plt.yticks([0., 30., 60., 90.], fontsize = 14)
        plt.ylabel("Latitude", fontsize = 16)
    else:
        plt.yticks([0., 30., 60., 90.], fontsize = 0)

    plt.xticks(fontsize = 0)

    ax1 = plt.subplot(5, 2, 7 + i)
    if i == 0:
        plt.title(labels[3],size=16, loc = "left")
    else:
        plt.title(labels2[3],size=16, loc = "left")

    if i == 0:
        C4 = plt.contourf( lon, lat, gradY / 100., levs, cmap = plt.cm.Reds)
    else:
        C4 = plt.contourf( lon, lat, gradY2 / 100., levs, cmap = plt.cm.Reds)

    plt.contour(lon, lat, land, levels=1,colors='k', linewidths = 0.5)
    plt.ylim([0., 90.])

    if i == 0:
        plt.yticks([0., 30., 60., 90.], fontsize = 14)
        plt.ylabel("Latitude", fontsize = 16)
    else:
        plt.yticks([0., 30., 60., 90.], fontsize = 0)

    plt.xticks(fontsize = 0)


    ax1 = plt.subplot(5, 2, 9 + i)
    if i == 0:
        plt.title(labels[4],size=16, loc = "left")
    else:
        plt.title(labels2[4],size=16, loc = "left")

    levs = np.arange(-12., 12.5, 0.5)
    if i == 0:
        C5 = plt.contourf( lon, lat, Zsub_press / 100., levs, cmap = plt.cm.coolwarm)
    else:
        C5 = plt.contourf( lon, lat, Zsub_press2 / 100., levs, cmap = plt.cm.coolwarm)

    plt.contour(lon, lat, land, levels=1,colors='k', linewidths = 0.5)
    if i == 0:
        Q = plt.quiver(lon[::8],lat[::4], Zsub_ucomp[::4,::8], Zsub_vcomp[::4,::8], scale = 60)
    else:
        Q = plt.quiver(lon[::8],lat[::4], Zsub_ucomp2[::4,::8], Zsub_vcomp2[::4,::8], scale = 60)

    ax1.quiverkey(Q, X=0.86, Y=1.05, U=2, label='2ms$^{-1}$', labelpos='E', fontproperties={'size': '12'})

    plt.ylim([0., 90.])

    plt.xticks([0., 60., 120., 180., 240., 300., 360.],fontsize = 14)

    plt.xlabel("Longitude", fontsize = 16)
    if i == 0:
        plt.ylabel("Latitude", fontsize = 16)
        plt.yticks([0., 30., 60., 90.], fontsize = 14)
    else:
        plt.yticks([0., 30., 60., 90.], fontsize = 0)
        
        
fig.tight_layout()

fig.subplots_adjust(right = 0.86)
cbar_ax = fig.add_axes([0.88, 0.815, 0.01, 0.16])
cb = fig.colorbar(C1, cax=cbar_ax, ticks = [10., 30., 50., 70., 90.])
cb.set_label( "Temperature var [K$^2$]", fontsize = 16  )
cb.ax.tick_params(labelsize=14)

cbar_ax2 = fig.add_axes([0.88, 0.621, 0.01, 0.16])
cb2 = fig.colorbar(C2, cax=cbar_ax2, ticks = [-6., -3., 0., 3., 6.])
cb2.set_label( "Temperature anom [K]", fontsize = 16  )
cb2.ax.tick_params(labelsize=14)

cbar_ax3 = fig.add_axes([0.88, 0.43, 0.01, 0.16])
cb3 = fig.colorbar(C3, cax=cbar_ax3, ticks = [0., 0.5, 1., 1.5])
cb3.set_label( r'$\left(\frac{\partial \overline{T}}{\partial x}\right)^2$' + " [(K/100km)$^2$]", fontsize = 16  )
cb3.ax.tick_params(labelsize=14)

cbar_ax4 = fig.add_axes([0.88, 0.234, 0.01, 0.16])
cb4 = fig.colorbar(C4, cax=cbar_ax4, ticks = [0., 0.5, 1., 1.5])
cb4.set_label( r'$\left(\frac{\partial \overline{T}}{\partial y}\right)^2$' + " [(K/100km)$^2$]", fontsize = 16  )
cb4.ax.tick_params(labelsize=14)

cbar_ax5 = fig.add_axes([0.88, 0.042, 0.01, 0.16])
cb5 = fig.colorbar(C5, cax=cbar_ax5, ticks = [-10., -5., 0., 5., 10.])
cb5.set_label( "Surf pressure anom [hPa]", fontsize = 16  )
cb5.ax.tick_params(labelsize=14)


plt.savefig("Narrow_Long.png")
plt.savefig("Narrow_Long.pdf")

In [ ]:
#Inland sea and GOM
var_temp, Zsub_temp, gradY, gradX, Zsub_press, u_850, v_850, Zsub_ucomp, Zsub_vcomp, lon, lat  = get_data( "GOM")
var_temp2, Zsub_temp2, gradY2, gradX2, Zsub_press2, u_8502, v_8502, Zsub_ucomp2, Zsub_vcomp2, lon, lat  = get_data( "Inland_Sea")

max_vars[3] = np.max(temp_var[90:]) #only look in NH
max_y[3] = np.max(gradY[90:] ** 2)
max_x[3] = np.max(gradX[90:] ** 2)
max_sum[3] = np.max(gradY[90:] ** 2 + gradX[90:] ** 2)

max_vars[4] = np.max(temp_var2[90:]) #only look in NH
max_y[4] = np.max(gradY2[90:] ** 2)
max_x[4] = np.max(gradX2[90:] ** 2)
max_sum[4] = np.max(gradY2[90:] ** 2 + gradX2[90:] ** 2)

In [ ]:
fig = plt.figure(figsize=(12,16))

labels = ["a) GOM", "c)", "e)", "g)", "i)"]
labels2 = ["b) Inland Sea", "d)", "f)", "h)", "j)"]

opt = ["GOM", "Inland_Sea"]
for i in range( 2 ):
    
    land = getland(opt[i])
    
    ax1 = plt.subplot(4, 2, 1 + i)
    if i == 0:
        plt.title(labels[0],size=18, loc = "left")
    else:
        plt.title(labels2[0],size=18, loc = "left")

    levs = np.arange(6., 99, 3.)
    if i == 0:
        C1 = plt.contourf( lon, lat, var_temp, levs, cmap = plt.cm.Reds)
    else:
        C1 = plt.contourf( lon, lat, var_temp2, levs, cmap = plt.cm.Reds)
        
    plt.contour(lon, lat, land, levels=1,colors='k', linewidths = 0.5)
    plt.ylim([0., 90.])

    if i == 0:
        plt.yticks([0., 30., 60., 90.], fontsize = 16)
        plt.ylabel("Latitude", fontsize = 17)
    else:
        plt.yticks([0., 30., 60., 90.], fontsize = 0)

    plt.xticks(fontsize = 0)


    if i == 0:
        Q = plt.quiver(lon[::8],lat[::4], u_850[::4,::8], v_850[::4,::8], scale = 120)
    else:
        Q = plt.quiver(lon[::8],lat[::4], u_8502[::4,::8], v_8502[::4,::8], scale = 120)

    ax1.quiverkey(Q, X=0.86, Y=1.05, U=5, label='5ms$^{-1}$', labelpos='E', fontproperties={'size': '12'})


    ax1 = plt.subplot(4, 2, 3 + i)
    if i == 0:
        plt.title(labels[1],size=18, loc = "left")
    else:
        plt.title(labels2[1],size=18, loc = "left")

    levs = np.arange(-8., 8.3, 0.3)
    if i == 0:
        C2 = plt.contourf( lon, lat, Zsub_temp, levs, cmap = plt.cm.coolwarm)
    else:
        C2 = plt.contourf( lon, lat, Zsub_temp2, levs, cmap = plt.cm.coolwarm)
        
    plt.contour(lon, lat, land, levels=1,colors='k', linewidths = 0.5)
    plt.ylim([0., 90.])

    if i == 0:
        plt.yticks([0., 30., 60., 90.], fontsize = 16)
        plt.ylabel("Latitude", fontsize = 17)
    else:
        plt.yticks([0., 30., 60., 90.], fontsize = 0)

    plt.xticks(fontsize = 0)

    levs = np.arange(0.2, 1.6, 0.1)
    ax1 = plt.subplot(4, 2, 5 + i)
    if i == 0:
        plt.title(labels[3],size=18, loc = "left")
    else:
        plt.title(labels2[3],size=18, loc = "left")

    if i == 0:
        C4 = plt.contourf( lon, lat, gradY / 100., levs, cmap = plt.cm.Reds)
    else:
        C4 = plt.contourf( lon, lat, gradY2 / 100., levs, cmap = plt.cm.Reds)

    plt.contour(lon, lat, land, levels=1,colors='k', linewidths = 0.5)
    plt.ylim([0., 90.])

    if i == 0:
        plt.yticks([0., 30., 60., 90.], fontsize = 16)
        plt.ylabel("Latitude", fontsize = 17)
    else:
        plt.yticks([0., 30., 60., 90.], fontsize = 0)

    plt.xticks(fontsize = 0)


    ax1 = plt.subplot(4, 2, 7 + i)
    if i == 0:
        plt.title(labels[4],size=18, loc = "left")
    else:
        plt.title(labels2[4],size=18, loc = "left")

    levs = np.arange(-12., 12.5, 0.5)
    if i == 0:
        C5 = plt.contourf( lon, lat, Zsub_press / 100., levs, cmap = plt.cm.coolwarm)
    else:
        C5 = plt.contourf( lon, lat, Zsub_press2 / 100., levs, cmap = plt.cm.coolwarm)

    plt.contour(lon, lat, land, levels=1,colors='k', linewidths = 0.5)
    if i == 0:
        Q = plt.quiver(lon[::8],lat[::4], Zsub_ucomp[::4,::8], Zsub_vcomp[::4,::8], scale = 60)
    else:
        Q = plt.quiver(lon[::8],lat[::4], Zsub_ucomp2[::4,::8], Zsub_vcomp2[::4,::8], scale = 60)

    ax1.quiverkey(Q, X=0.86, Y=1.05, U=2, label='2ms$^{-1}$', labelpos='E', fontproperties={'size': '12'})

    plt.ylim([0., 90.])

    plt.xticks([0., 60., 120., 180., 240., 300., 360.],fontsize = 16)

    plt.xlabel("Longitude", fontsize = 17)
    if i == 0:
        plt.ylabel("Latitude", fontsize = 17)
        plt.yticks([0., 30., 60., 90.], fontsize = 16)
    else:
        plt.yticks([0., 30., 60., 90.], fontsize = 0)
        
        
fig.tight_layout()

fig.subplots_adjust(right = 0.86)
cbar_ax = fig.add_axes([0.88, 0.773, 0.01, 0.19])
cb = fig.colorbar(C1, cax=cbar_ax, ticks = [10., 30., 50., 70., 90.])
cb.set_label( "Temperature var\n [K$^2$]", fontsize = 18  )
cb.ax.tick_params(labelsize=14)

cbar_ax2 = fig.add_axes([0.88, 0.537, 0.01, 0.19])
cb2 = fig.colorbar(C2, cax=cbar_ax2, ticks = [-6., -3., 0., 3., 6.])
cb2.set_label( "Temperature anom\n [K]", fontsize = 18  )
cb2.ax.tick_params(labelsize=14)

cbar_ax4 = fig.add_axes([0.88, 0.3, 0.01, 0.19])
cb4 = fig.colorbar(C4, cax=cbar_ax4, ticks = [0., 0.5, 1., 1.5])
cb4.set_label( r'$\left(\frac{\partial \overline{T}}{\partial y}\right)^2$' + " [(K/100km)$^2$]", fontsize = 18  )
cb4.ax.tick_params(labelsize=14)

cbar_ax5 = fig.add_axes([0.88, 0.064, 0.01, 0.19])
cb5 = fig.colorbar(C5, cax=cbar_ax5, ticks = [-10., -5., 0., 5., 10.])
cb5.set_label( "Surf pressure\n anom [hPa]", fontsize = 18  )
cb5.ax.tick_params(labelsize=14)


plt.savefig("Inland_Sea_GOM.png")
plt.savefig("Inland_Sea_GOM.pdf")

In [ ]:
#Slanted and Tapered
var_temp, Zsub_temp, gradY, gradX, Zsub_press, u_850, v_850, Zsub_ucomp, Zsub_vcomp, lon, lat  = get_data( "Slanted")
var_temp2, Zsub_temp2, gradY2, gradX2, Zsub_press2, u_8502, v_8502, Zsub_ucomp2, Zsub_vcomp2, lon, lat  = get_data( "Tapered")

max_vars[5] = np.max(temp_var[90:]) #only look in NH
max_y[5] = np.max(gradY[90:] ** 2)
max_x[5] = np.max(gradX[90:] ** 2)
max_sum[5] = np.max(gradY[90:] ** 2 + gradX[90:] ** 2)

max_vars[6] = np.max(temp_var2[90:]) #only look in NH
max_y[6] = np.max(gradY2[90:] ** 2)
max_x[6] = np.max(gradX2[90:] ** 2)
max_sum[6] = np.max(gradY2[90:] ** 2 + gradX2[90:] ** 2)

In [ ]:
fig = plt.figure(figsize=(12,16))

labels = ["a) Slanted", "c)", "e)", "g)", "i)"]
labels2 = ["b) Tapered", "d)", "f)", "h)", "j)"]

opt = ["Slanted", "Tapered"]
for i in range( 2 ):
    
    land = getland(opt[i])
    
    ax1 = plt.subplot(5, 2, 1 + i)
    if i == 0:
        plt.title(labels[0],size=16, loc = "left")
    else:
        plt.title(labels2[0],size=16, loc = "left")

    levs = np.arange(6., 99, 3.)
    if i == 0:
        C1 = plt.contourf( lon, lat, var_temp, levs, cmap = plt.cm.Reds)
    else:
        C1 = plt.contourf( lon, lat, var_temp2, levs, cmap = plt.cm.Reds)
        
    plt.contour(lon, lat, land, levels=1,colors='k', linewidths = 0.5)
    plt.ylim([0., 90.])

    if i == 0:
        plt.yticks([0., 30., 60., 90.], fontsize = 14)
        plt.ylabel("Latitude", fontsize = 16)
    else:
        plt.yticks([0., 30., 60., 90.], fontsize = 0)

    plt.xticks(fontsize = 0)


    if i == 0:
        Q = plt.quiver(lon[::8],lat[::4], u_850[::4,::8], v_850[::4,::8], scale = 120)
    else:
        Q = plt.quiver(lon[::8],lat[::4], u_8502[::4,::8], v_8502[::4,::8], scale = 120)

    ax1.quiverkey(Q, X=0.86, Y=1.05, U=5, label='5ms$^{-1}$', labelpos='E', fontproperties={'size': '12'})


    ax1 = plt.subplot(5, 2, 3 + i)
    if i == 0:
        plt.title(labels[1],size=16, loc = "left")
    else:
        plt.title(labels2[1],size=16, loc = "left")

    levs = np.arange(-8., 8.3, 0.3)
    if i == 0:
        C2 = plt.contourf( lon, lat, Zsub_temp, levs, cmap = plt.cm.coolwarm)
    else:
        C2 = plt.contourf( lon, lat, Zsub_temp2, levs, cmap = plt.cm.coolwarm)
        
    plt.contour(lon, lat, land, levels=1,colors='k', linewidths = 0.5)
    plt.ylim([0., 90.])

    if i == 0:
        plt.yticks([0., 30., 60., 90.], fontsize = 14)
        plt.ylabel("Latitude", fontsize = 16)
    else:
        plt.yticks([0., 30., 60., 90.], fontsize = 0)

    plt.xticks(fontsize = 0)

    ax1 = plt.subplot(5, 2, 5 + i)
    if i == 0:
        plt.title(labels[2],size=16, loc = "left")
    else:
        plt.title(labels2[2],size=16, loc = "left")

    levs = np.arange(0.2, 1.6, 0.1)
    if i == 0:
        C3 = plt.contourf( lon, lat, gradX / 100., levs, cmap = plt.cm.Reds)
    else:
        C3 = plt.contourf( lon, lat, gradX2 / 100., levs, cmap = plt.cm.Reds)
        
    plt.contour(lon, lat, land, levels=1,colors='k', linewidths = 0.5)
    plt.ylim([0., 90.])

    if i == 0:
        plt.yticks([0., 30., 60., 90.], fontsize = 14)
        plt.ylabel("Latitude", fontsize = 16)
    else:
        plt.yticks([0., 30., 60., 90.], fontsize = 0)

    plt.xticks(fontsize = 0)

    ax1 = plt.subplot(5, 2, 7 + i)
    if i == 0:
        plt.title(labels[3],size=16, loc = "left")
    else:
        plt.title(labels2[3],size=16, loc = "left")

    if i == 0:
        C4 = plt.contourf( lon, lat, gradY / 100., levs, cmap = plt.cm.Reds)
    else:
        C4 = plt.contourf( lon, lat, gradY2 / 100., levs, cmap = plt.cm.Reds)

    plt.contour(lon, lat, land, levels=1,colors='k', linewidths = 0.5)
    plt.ylim([0., 90.])

    if i == 0:
        plt.yticks([0., 30., 60., 90.], fontsize = 14)
        plt.ylabel("Latitude", fontsize = 16)
    else:
        plt.yticks([0., 30., 60., 90.], fontsize = 0)

    plt.xticks(fontsize = 0)


    ax1 = plt.subplot(5, 2, 9 + i)
    if i == 0:
        plt.title(labels[4],size=16, loc = "left")
    else:
        plt.title(labels2[4],size=16, loc = "left")

    levs = np.arange(-12., 12.5, 0.5)
    if i == 0:
        C5 = plt.contourf( lon, lat, Zsub_press / 100., levs, cmap = plt.cm.coolwarm)
    else:
        C5 = plt.contourf( lon, lat, Zsub_press2 / 100., levs, cmap = plt.cm.coolwarm)

    plt.contour(lon, lat, land, levels=1,colors='k', linewidths = 0.5)
    if i == 0:
        Q = plt.quiver(lon[::8],lat[::4], Zsub_ucomp[::4,::8], Zsub_vcomp[::4,::8], scale = 60)
    else:
        Q = plt.quiver(lon[::8],lat[::4], Zsub_ucomp2[::4,::8], Zsub_vcomp2[::4,::8], scale = 60)

    ax1.quiverkey(Q, X=0.86, Y=1.05, U=2, label='2ms$^{-1}$', labelpos='E', fontproperties={'size': '12'})

    plt.ylim([0., 90.])

    plt.xticks([0., 60., 120., 180., 240., 300., 360.],fontsize = 14)

    plt.xlabel("Longitude", fontsize = 16)
    if i == 0:
        plt.ylabel("Latitude", fontsize = 16)
        plt.yticks([0., 30., 60., 90.], fontsize = 14)
    else:
        plt.yticks([0., 30., 60., 90.], fontsize = 0)
        
        
fig.tight_layout()

fig.subplots_adjust(right = 0.86)
cbar_ax = fig.add_axes([0.88, 0.815, 0.01, 0.16])
cb = fig.colorbar(C1, cax=cbar_ax, ticks = [10., 30., 50., 70., 90.])
cb.set_label( "Temperature var [K$^2$]", fontsize = 16  )
cb.ax.tick_params(labelsize=14)

cbar_ax2 = fig.add_axes([0.88, 0.621, 0.01, 0.16])
cb2 = fig.colorbar(C2, cax=cbar_ax2, ticks = [-6., -3., 0., 3., 6.])
cb2.set_label( "Temperature anom [K]", fontsize = 16  )
cb2.ax.tick_params(labelsize=14)

cbar_ax3 = fig.add_axes([0.88, 0.43, 0.01, 0.16])
cb3 = fig.colorbar(C3, cax=cbar_ax3, ticks = [0., 0.5, 1., 1.5])
cb3.set_label( r'$\left(\frac{\partial \overline{T}}{\partial x}\right)^2$' + " [(K/100km)$^2$]", fontsize = 16  )
cb3.ax.tick_params(labelsize=14)

cbar_ax4 = fig.add_axes([0.88, 0.234, 0.01, 0.16])
cb4 = fig.colorbar(C4, cax=cbar_ax4, ticks = [0., 0.5, 1., 1.5])
cb4.set_label( r'$\left(\frac{\partial \overline{T}}{\partial y}\right)^2$' + " [(K/100km)$^2$]", fontsize = 16  )
cb4.ax.tick_params(labelsize=14)

cbar_ax5 = fig.add_axes([0.88, 0.042, 0.01, 0.16])
cb5 = fig.colorbar(C5, cax=cbar_ax5, ticks = [-10., -5., 0., 5., 10.])
cb5.set_label( "Surf pressure anom [hPa]", fontsize = 16  )
cb5.ax.tick_params(labelsize=14)


plt.savefig("Slanted_Tapered.png")
plt.savefig("Slanted_Tapered.pdf")

In [ ]:
#Two continents
var_temp, Zsub_temp, gradY, gradX, Zsub_press, u_850, v_850, Zsub_ucomp, Zsub_vcomp, lon, lat  = get_data( "two_continents")
var_temp2, Zsub_temp2, gradY2, gradX2, Zsub_press2, u_8502, v_8502, Zsub_ucomp2, Zsub_vcomp2, lon, lat  = get_data( "two_tapered")

max_vars[7] = np.max(temp_var[90:, :128]) #broad continent
max_y[7] = np.max(gradY[90:, :128] ** 2)
max_x[7] = np.max(gradX[90:, :128] ** 2)
max_sum[7] = np.max(gradY[90:, :128] ** 2 + gradX[90:, :128] ** 2)

max_vars[8] = np.max(temp_var2[90:, :128]) 
max_y[8] = np.max(gradY2[90:, :128] ** 2)
max_x[8] = np.max(gradX2[90:, :128] ** 2)
max_sum[8] = np.max(gradY2[90:, :128] ** 2 + gradX2[90:, :128] ** 2)

max_vars[7] = np.max(temp_var[90:, 128:]) #narrow continent
max_y[7] = np.max(gradY[90:, 128:] ** 2)
max_x[7] = np.max(gradX[90:, 128:] ** 2)
max_sum[7] = np.max(gradY[90:, 128:] ** 2 + gradX[90:, 128:] ** 2)

max_vars[8] = np.max(temp_var2[90:, 128:]) 
max_y[8] = np.max(gradY2[90:, 128:] ** 2)
max_x[8] = np.max(gradX2[90:, 128:] ** 2)
max_sum[8] = np.max(gradY2[90:, 128:] ** 2 + gradX2[90:, 128:] ** 2)

In [ ]:
fig = plt.figure(figsize=(12,16))

labels = ["a) Two continents", "c)", "e)", "g)", "i)"]
labels2 = ["b) Two continents + tapering", "d)", "f)", "h)", "j)"]

opt = ["two_continents", "two_tapered"]
for i in range( 2 ):
    
    land = getland(opt[i])
    
    ax1 = plt.subplot(5, 2, 1 + i)
    if i == 0:
        plt.title(labels[0],size=16, loc = "left")
    else:
        plt.title(labels2[0],size=16, loc = "left")

    levs = np.arange(6., 99, 3.)
    if i == 0:
        C1 = plt.contourf( lon, lat, var_temp, levs, cmap = plt.cm.Reds)
    else:
        C1 = plt.contourf( lon, lat, var_temp2, levs, cmap = plt.cm.Reds)
        
    plt.contour(lon, lat, land, levels=1,colors='k', linewidths = 0.5)
    plt.ylim([0., 90.])

    if i == 0:
        plt.yticks([0., 30., 60., 90.], fontsize = 14)
        plt.ylabel("Latitude", fontsize = 16)
    else:
        plt.yticks([0., 30., 60., 90.], fontsize = 0)

    plt.xticks(fontsize = 0)


    if i == 0:
        Q = plt.quiver(lon[::8],lat[::4], u_850[::4,::8], v_850[::4,::8], scale = 120)
    else:
        Q = plt.quiver(lon[::8],lat[::4], u_8502[::4,::8], v_8502[::4,::8], scale = 120)

    ax1.quiverkey(Q, X=0.86, Y=1.05, U=5, label='5ms$^{-1}$', labelpos='E', fontproperties={'size': '12'})


    ax1 = plt.subplot(5, 2, 3 + i)
    if i == 0:
        plt.title(labels[1],size=16, loc = "left")
    else:
        plt.title(labels2[1],size=16, loc = "left")

    levs = np.arange(-8., 8.3, 0.3)
    if i == 0:
        C2 = plt.contourf( lon, lat, Zsub_temp, levs, cmap = plt.cm.coolwarm)
    else:
        C2 = plt.contourf( lon, lat, Zsub_temp2, levs, cmap = plt.cm.coolwarm)
        
    plt.contour(lon, lat, land, levels=1,colors='k', linewidths = 0.5)
    plt.ylim([0., 90.])

    if i == 0:
        plt.yticks([0., 30., 60., 90.], fontsize = 14)
        plt.ylabel("Latitude", fontsize = 16)
    else:
        plt.yticks([0., 30., 60., 90.], fontsize = 0)

    plt.xticks(fontsize = 0)

    ax1 = plt.subplot(5, 2, 5 + i)
    if i == 0:
        plt.title(labels[2],size=16, loc = "left")
    else:
        plt.title(labels2[2],size=16, loc = "left")

    levs = np.arange(0.2, 1.6, 0.1)
    if i == 0:
        C3 = plt.contourf( lon, lat, gradX / 100., levs, cmap = plt.cm.Reds)
    else:
        C3 = plt.contourf( lon, lat, gradX2 / 100., levs, cmap = plt.cm.Reds)
        
    plt.contour(lon, lat, land, levels=1,colors='k', linewidths = 0.5)
    plt.ylim([0., 90.])

    if i == 0:
        plt.yticks([0., 30., 60., 90.], fontsize = 14)
        plt.ylabel("Latitude", fontsize = 16)
    else:
        plt.yticks([0., 30., 60., 90.], fontsize = 0)

    plt.xticks(fontsize = 0)

    ax1 = plt.subplot(5, 2, 7 + i)
    if i == 0:
        plt.title(labels[3],size=16, loc = "left")
    else:
        plt.title(labels2[3],size=16, loc = "left")

    if i == 0:
        C4 = plt.contourf( lon, lat, gradY / 100., levs, cmap = plt.cm.Reds)
    else:
        C4 = plt.contourf( lon, lat, gradY2 / 100., levs, cmap = plt.cm.Reds)

    plt.contour(lon, lat, land, levels=1,colors='k', linewidths = 0.5)
    plt.ylim([0., 90.])

    if i == 0:
        plt.yticks([0., 30., 60., 90.], fontsize = 14)
        plt.ylabel("Latitude", fontsize = 16)
    else:
        plt.yticks([0., 30., 60., 90.], fontsize = 0)

    plt.xticks(fontsize = 0)


    ax1 = plt.subplot(5, 2, 9 + i)
    if i == 0:
        plt.title(labels[4],size=16, loc = "left")
    else:
        plt.title(labels2[4],size=16, loc = "left")

    levs = np.arange(-12., 12.5, 0.5)
    if i == 0:
        C5 = plt.contourf( lon, lat, Zsub_press / 100., levs, cmap = plt.cm.coolwarm)
    else:
        C5 = plt.contourf( lon, lat, Zsub_press2 / 100., levs, cmap = plt.cm.coolwarm)

    plt.contour(lon, lat, land, levels=1,colors='k', linewidths = 0.5)
    if i == 0:
        Q = plt.quiver(lon[::8],lat[::4], Zsub_ucomp[::4,::8], Zsub_vcomp[::4,::8], scale = 60)
    else:
        Q = plt.quiver(lon[::8],lat[::4], Zsub_ucomp2[::4,::8], Zsub_vcomp2[::4,::8], scale = 60)

    ax1.quiverkey(Q, X=0.86, Y=1.05, U=2, label='2ms$^{-1}$', labelpos='E', fontproperties={'size': '12'})

    plt.ylim([0., 90.])

    plt.xticks([0., 60., 120., 180., 240., 300., 360.],fontsize = 14)

    plt.xlabel("Longitude", fontsize = 16)
    if i == 0:
        plt.ylabel("Latitude", fontsize = 16)
        plt.yticks([0., 30., 60., 90.], fontsize = 14)
    else:
        plt.yticks([0., 30., 60., 90.], fontsize = 0)
        
        
fig.tight_layout()

fig.subplots_adjust(right = 0.86)
cbar_ax = fig.add_axes([0.88, 0.815, 0.01, 0.16])
cb = fig.colorbar(C1, cax=cbar_ax, ticks = [10., 30., 50., 70., 90.])
cb.set_label( "Temperature var [K$^2$]", fontsize = 16  )
cb.ax.tick_params(labelsize=14)

cbar_ax2 = fig.add_axes([0.88, 0.621, 0.01, 0.16])
cb2 = fig.colorbar(C2, cax=cbar_ax2, ticks = [-6., -3., 0., 3., 6.])
cb2.set_label( "Temperature anom [K]", fontsize = 16  )
cb2.ax.tick_params(labelsize=14)

cbar_ax3 = fig.add_axes([0.88, 0.43, 0.01, 0.16])
cb3 = fig.colorbar(C3, cax=cbar_ax3, ticks = [0., 0.5, 1., 1.5])
cb3.set_label( r'$\left(\frac{\partial \overline{T}}{\partial x}\right)^2$' + " [(K/100km)$^2$]", fontsize = 16  )
cb3.ax.tick_params(labelsize=14)

cbar_ax4 = fig.add_axes([0.88, 0.234, 0.01, 0.16])
cb4 = fig.colorbar(C4, cax=cbar_ax4, ticks = [0., 0.5, 1., 1.5])
cb4.set_label( r'$\left(\frac{\partial \overline{T}}{\partial y}\right)^2$' + " [(K/100km)$^2$]", fontsize = 16  )
cb4.ax.tick_params(labelsize=14)

cbar_ax5 = fig.add_axes([0.88, 0.042, 0.01, 0.16])
cb5 = fig.colorbar(C5, cax=cbar_ax5, ticks = [-10., -5., 0., 5., 10.])
cb5.set_label( "Surf pressure anom [hPa]", fontsize = 16  )
cb5.ax.tick_params(labelsize=14)


plt.savefig("two_Continents.png")
plt.savefig("two_Continents.pdf")

Scatter plot of maximum variances vs gradients. Slope gives effective mixing lengths

In [ ]:
#Just do maximum meridional gradients:
z = np.poly1d(np.polyfit(np.array(max_y) / 100., max_vars, 1))
z2 = np.poly1d(np.polyfit(max_vars, np.array(max_y) / 100., 1))

sl, inter, r, stds, stdi = regress2(np.array(max_y) / 100., max_vars, z[1], z[0], z2[1], z2[0], _method_type_2 = "reduced major axis")
#print effective mixing length and r^2 value
print(np.sqrt(sl), r ** 2)

#Maximum total gradients:
z = np.poly1d(np.polyfit(np.array(max_sum) / 100., max_vars, 1))
z2 = np.poly1d(np.polyfit(max_vars, np.array(max_sum) / 100., 1))

sl2, inter2, r2, stds, stdi = regress2(np.array(max_sum) / 100., max_vars, z[1], z[0], z2[1], z2[0], _method_type_2 = "reduced major axis")    
#print effective mixing length and r^2 value
print(np.sqrt(sl2), r2 ** 2)

fig = plt.figure(figsize=(7,5))
fig.subplots_adjust(right = 0.86, top = 0.97, left = 0.14, bottom = 0.12,  wspace = 0.3, hspace = 0.45)

ax = plt.subplot(1, 1, 1)
plt.plot(np.array(max_y) / 100., max_vars, 'ko')
plt.plot(np.array(max_sum) / 100., max_vars, 'kx')

plt.legend(["$(\partial T / \partial y)^2$", "$(\partial T / \partial y)^2 + (\partial T / \partial x)^2$"],
          frameon = True, bbox_to_anchor = (.75, 0.5), fontsize = 12 )

z = np.poly1d(np.polyfit(np.array(max_y) / 100., max_vars, 1))
z2 = np.poly1d(np.polyfit(np.array(max_sum) / 100., max_vars, 1))
x = np.linspace(0., 4., 100)
plt.plot(x, sl * x + inter, 'k', linewidth = 0.75)
plt.plot(x, sl2 * x + inter2, 'k--', linewidth = 0.75)

plt.ylim([57., 90.])
plt.xlim([0., 2.2])

plt.xlabel("maximum squared temperature gradient [(K / 100km)$^2$]", fontsize = 14)
plt.ylabel("maximum temperature variance [K$^2$]", fontsize = 14)

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.xticks([0., 0.5, 1., 1.5, 2.], fontsize = 12)
plt.yticks(fontsize = 12)

plt.savefig("Effective_mixing_lengths.png")
plt.savefig("Effective_mixing_lengths.pdf")

Now do budgets:

In [ ]:
e_r = 6.371 * 10 ** 6

def diff_lat( dat, y ):
        l1 = len(dat)
        del_dat = np.zeros( l1 )

        for i in range( l1 ):
                if i == 0:
                        del_dat[i] = ( dat[1] - dat[0] ) / (y[1] - y[0] )
                elif i == l1 - 1:
                        del_dat[i] = ( dat[l1 - 1] - dat[l1 - 2] ) / (y[l1 - 1] - y[l1 - 2] )
                else:
                        del_dat[i] = ( dat[i + 1] - dat[i - 1] ) / (y[i + 1] - y[i - 1] )

        return del_dat / e_r
    
def diff_lat2( dat, y ):
        l1, l2 = np.shape(dat)
        del_dat = np.zeros( ( l1, l2) )

        for i in range( l1 ):
                if i == 0:
                        del_dat[i, :] = ( dat[1, :] - dat[0, :] ) / (y[1] - y[0] )
                elif i == l1 - 1:
                        del_dat[i, :] = ( dat[l1 - 1, :] - dat[l1 - 2, :] ) / (y[l1 - 1] - y[l1 - 2] )
                else:
                        del_dat[i, :] = ( dat[i + 1, :] - dat[i - 1, :] ) / (y[i + 1] - y[i - 1] )

        return del_dat / e_r

def diff_lon( dat, x ):
        l1, l2 = np.shape(dat)
        del_dat = np.zeros( ( l1, l2) )

        for i in range( l2 ):
                if i == 0:
                        del_dat[:, i] = ( dat[:, 1] - dat[ :, l2 - 1] ) / (x[1] - x[l2 - 1])
                elif i == l2 - 1:
                        del_dat[:, i] = ( dat[:, 0] - dat[:, l2 - 2] ) / (x[0] - x[l2 - 2] )
                else:
                        del_dat[:, i] = ( dat[:, i + 1] - dat[:, i - 1] ) / (x[i + 1] - x[i - 1])

        return del_dat / e_r
    
def divergence( u, v, x, y ):
        l = len(u)
        j = len(u[0])
        div = np.zeros( ( l, j ) )

        rad_x = x * np.pi / 180.
        rad_y = y * np.pi / 180.


        for i in range( l ):
                div[i, :] = diff_lon( u[i], rad_x ) / np.cos( rad_y[i] )
        for i in range( j ):
                div[:, i] += diff_lat( v[:, i] * np.cos( rad_y ), rad_y ) / np.cos( rad_y )

        return div
        
def calc_temp_adv( opt ):
    
    data=getdata(opt)
    t_850 = data['temp'].interp(pfull=850)[:]
    wint_temp_850 = np.zeros( ( (( 15, 90, 128, 256))))

    a = 675
    for i in range( 15):
        wint_temp_850[i, :, :, :] = t_850[a: a + 90, :, :]
        print(i)
        a += 360
    temp_850 = np.mean( np.mean(wint_temp_850, axis = 0), axis = 0)
    u_850 = data['ucomp'].interp(pfull=850)[:]
    v_850 = data['vcomp'].interp(pfull=850)[:]

    wint_u_850 = np.zeros( ( (( 15, 90, 128, 256))))
    wint_v_850 = np.zeros( ( (( 15, 90, 128, 256))))

    a = 675
    for i in range( 15):
        wint_u_850[i, :, :, :] = u_850[a: a + 90, :, :]
        wint_v_850[i, :, :, :] = v_850[a: a + 90, :, :]
        print(i)
        a += 360
    u_850 = np.mean( np.mean(wint_u_850, axis = 0), axis = 0)
    v_850 = np.mean( np.mean(wint_v_850, axis = 0), axis = 0)

    #Have data, now get eddy and mean flows
    eddy_T = temp_850 - np.mean(temp_850, axis = 1)[:, np.newaxis]
    eddy_u = u_850 - np.mean(u_850, axis = 1)[:, np.newaxis]
    eddy_v = v_850 - np.mean(v_850, axis = 1)[:, np.newaxis]

    mean_T = np.mean(temp_850, axis = 1)
    mean_u = np.mean(u_850, axis = 1)
    mean_v = np.mean(v_850, axis = 1)

    #Now calculate budget terms:
    lon = np.array(data['lon'][:])
    lat = np.array(data['lat'][:])
    rad_x = lon * np.pi / 180.
    rad_y = lat * np.pi / 180.
    
    mflow = mean_u[:, np.newaxis] * diff_lon( eddy_T, rad_x) +  mean_v[:, np.newaxis] * diff_lat2( eddy_T, rad_y)
    eflow = eddy_v * diff_lat( mean_T, rad_y)[:, np.newaxis]
    nlflow = eddy_u * diff_lon( eddy_T, rad_x) +  eddy_v * diff_lat2( eddy_T, rad_y)

    return mflow, eflow, nlflow

In [ ]:
#Start with control:
mflow, eflow, nlflow = calc_temp_adv( "control" )

In [ ]:
fig = plt.figure(figsize=(22,5))
fig.subplots_adjust(top = 0.9, left = 0.07, bottom = 0.15,  wspace = 0.2, hspace = 0.45)

v = np.linspace(-3., 3., 20)
plt.subplot(1, 3, 1)
plt.title('a) '+r'$-\overline{[\mathbf{v}]}\cdot\nabla \overline{T^*}$', fontsize = 20)

plt.contourf(lon, lat, -mflow * 86400., v, cmap = plt.cm.seismic)
plt.ylim([10., 90.])
levs = [-1000., 1., 1000.]
plt.contour(lon, lat,land, levs, colors = 'k')
plt.xlabel("Longitude", fontsize = 20)
plt.ylabel("Latitude", fontsize = 20)
plt.xticks([0., 60., 120., 180., 240., 300., 360.], fontsize = 18)
plt.yticks([10., 30., 50., 70., 90.], fontsize = 18)


ax = plt.subplot(1, 3, 2)
plt.title('b) '+r'$-\overline{\mathbf{v}^*}\cdot\nabla \overline{[T]}$', fontsize = 20)

plt.contourf(lon, lat, -eflow * 86400., v, cmap = plt.cm.seismic)
plt.ylim([10., 90.])
plt.contour(lon, lat,land, levs, colors = 'k')
plt.xlabel("Longitude", fontsize = 20)
#plt.ylabel("Latitude", fontsize = 20)
plt.xticks([0., 60., 120., 180., 240., 300., 360.], fontsize = 18)
plt.yticks(fontsize = 0)

ax = plt.subplot(1, 3, 3)
plt.title('c) '+r'$-\overline{\mathbf{v}^*}\cdot\nabla \overline{T^*}$', fontsize = 20)

im = plt.contourf(lon, lat, -nlflow * 86400., v, cmap = plt.cm.seismic)
plt.ylim([10., 90.])
plt.contour(lon, lat,land, levs, colors = 'k')
plt.xlabel("Longitude", fontsize = 20)
#plt.ylabel("Latitude", fontsize = 20)
plt.xticks([0., 60., 120., 180., 240., 300., 360.], fontsize = 18)
plt.yticks(fontsize = 0)


cbar_ax = fig.add_axes([0.92, 0.15, 0.011, 0.74])
cb = fig.colorbar(im, cax=cbar_ax, ticks = [-3., -1.5, 0., 1.5, 3.])
cb.set_label( "[K / day]", fontsize = 20  )
cb.ax.tick_params(labelsize=16) 
plt.savefig("control_land_temp_advection2.png");
plt.savefig("control_land_temp_advection2.pdf");

In [ ]:
#Narrow/long
mflow, eflow, nlflow = calc_temp_adv( "narrow" )
mflow2, eflow2, nlflow2 = calc_temp_adv( "long" )

In [ ]:
fig = plt.figure(figsize=(22,9))
fig.subplots_adjust(top = 0.9, left = 0.07, bottom = 0.15,  wspace = 0.15, hspace = 0.3)

land = getland("narrow")
land2 = getland("long")
    
    
v = np.linspace(-3., 3., 20)
plt.subplot(2, 3, 1)
plt.title('a) '+r'$-\overline{[\mathbf{v}]}\cdot\nabla \overline{T^*}$', fontsize = 22)

plt.contourf(lon, lat, -mflow * 86400., v, cmap = plt.cm.seismic)
plt.ylim([10., 90.])
levs = [-1000., 1., 1000.]
plt.contour(lon, lat,land, levs, colors = 'k')
#plt.xlabel("Longitude", fontsize = 20)
plt.ylabel("Latitude", fontsize = 20)
plt.xticks([0., 60., 120., 180., 240., 300., 360.], fontsize = 0)
plt.yticks([10., 30., 50., 70., 90.], fontsize = 18)


ax = plt.subplot(2, 3, 2)
plt.title('b) '+r'$-\overline{\mathbf{v}^*}\cdot\nabla \overline{[T]}$', fontsize = 22)

plt.contourf(lon, lat, -eflow * 86400., v, cmap = plt.cm.seismic)
plt.ylim([10., 90.])
plt.contour(lon, lat,land, levs, colors = 'k')
#plt.xlabel("Longitude", fontsize = 20)
#plt.ylabel("Latitude", fontsize = 20)
plt.xticks([0., 60., 120., 180., 240., 300., 360.], fontsize = 0)
plt.yticks([10., 30., 50., 70., 90.], fontsize = 0)

ax = plt.subplot(2, 3, 3)
plt.title('c) '+r'$-\overline{\mathbf{v}^*}\cdot\nabla \overline{T^*}$', fontsize = 22)

im = plt.contourf(lon, lat, -nlflow * 86400., v, cmap = plt.cm.seismic)
plt.ylim([10., 90.])
plt.contour(lon, lat,land, levs, colors = 'k')
#plt.xlabel("Longitude", fontsize = 20)
#plt.ylabel("Latitude", fontsize = 20)
plt.xticks([0., 60., 120., 180., 240., 300., 360.], fontsize = 0)
plt.yticks([10., 30., 50., 70., 90.], fontsize = 0)

plt.subplot(2, 3, 4)
plt.title("d)", fontsize = 22)

plt.contourf(lon, lat, -mflow2 * 86400., v, cmap = plt.cm.seismic)
plt.ylim([10., 90.])
levs = [-1000., 1., 1000.]
plt.contour(lon, lat,land2, levs, colors = 'k')
plt.xlabel("Longitude", fontsize = 20)
plt.ylabel("Latitude", fontsize = 20)
plt.xticks([0., 60., 120., 180., 240., 300., 360.], fontsize = 18)
plt.yticks([10., 30., 50., 70., 90.], fontsize = 18)


ax = plt.subplot(2, 3, 5)
plt.title("e)", fontsize = 22)

plt.contourf(lon, lat, -eflow2 * 86400., v, cmap = plt.cm.seismic)
plt.ylim([10., 90.])
plt.contour(lon, lat,land2, levs, colors = 'k')
plt.xlabel("Longitude", fontsize = 20)
#plt.ylabel("Latitude", fontsize = 20)
plt.xticks([0., 60., 120., 180., 240., 300., 360.], fontsize = 18)
plt.yticks([10., 30., 50., 70., 90.], fontsize = 0)

ax = plt.subplot(2, 3, 6)
plt.title("f)", fontsize = 22)

im = plt.contourf(lon, lat, -nlflow * 86400., v, cmap = plt.cm.seismic)
plt.ylim([10., 90.])
plt.contour(lon, lat,land2, levs, colors = 'k')
plt.xlabel("Longitude", fontsize = 20)
#plt.ylabel("Latitude", fontsize = 20)
plt.xticks([0., 60., 120., 180., 240., 300., 360.], fontsize = 18)
plt.yticks([10., 30., 50., 70., 90.], fontsize = 0)


cbar_ax = fig.add_axes([0.92, 0.15, 0.011, 0.74])
cb = fig.colorbar(im, cax=cbar_ax, ticks = [-3., -1.5, 0., 1.5, 3.])
cb.set_label( "[K/day]", fontsize = 22  )
cb.ax.tick_params(labelsize=18) 
plt.savefig("narrow_wide_temp_advection.png")
plt.savefig("narrow_wide_temp_advection.pdf")

In [ ]:
#GOM/Inland Sea
mflow, eflow, nlflow = calc_temp_adv( "GOM" )
mflow2, eflow2, nlflow2 = calc_temp_adv( "Inland_Sea" )

In [ ]:
fig = plt.figure(figsize=(22,9))
fig.subplots_adjust(top = 0.9, left = 0.07, bottom = 0.15,  wspace = 0.15, hspace = 0.3)

opt = ["GOM", "Inland_Sea"]
land = getland(opt[0])
land2 = getland(opt[1])
    
    
v = np.linspace(-3., 3., 20)
plt.subplot(2, 3, 1)
plt.title('a) '+r'$-\overline{[\mathbf{v}]}\cdot\nabla \overline{T^*}$', fontsize = 22)

plt.contourf(lon, lat, -mflow * 86400., v, cmap = plt.cm.seismic)
plt.ylim([10., 90.])
levs = [-1000., 1., 1000.]
plt.contour(lon, lat,land, levs, colors = 'k')
#plt.xlabel("Longitude", fontsize = 20)
plt.ylabel("Latitude", fontsize = 20)
plt.xticks([0., 60., 120., 180., 240., 300., 360.], fontsize = 0)
plt.yticks([10., 30., 50., 70., 90.], fontsize = 18)


ax = plt.subplot(2, 3, 2)
plt.title('b) '+r'$-\overline{\mathbf{v}^*}\cdot\nabla \overline{[T]}$', fontsize = 22)

plt.contourf(lon, lat, -eflow * 86400., v, cmap = plt.cm.seismic)
plt.ylim([10., 90.])
plt.contour(lon, lat,land, levs, colors = 'k')
#plt.xlabel("Longitude", fontsize = 20)
#plt.ylabel("Latitude", fontsize = 20)
plt.xticks([0., 60., 120., 180., 240., 300., 360.], fontsize = 0)
plt.yticks([10., 30., 50., 70., 90.], fontsize = 0)

ax = plt.subplot(2, 3, 3)
plt.title('c) '+r'$-\overline{\mathbf{v}^*}\cdot\nabla \overline{T^*}$', fontsize = 22)

im = plt.contourf(lon, lat, -nlflow * 86400., v, cmap = plt.cm.seismic)
plt.ylim([10., 90.])
plt.contour(lon, lat,land, levs, colors = 'k')
#plt.xlabel("Longitude", fontsize = 20)
#plt.ylabel("Latitude", fontsize = 20)
plt.xticks([0., 60., 120., 180., 240., 300., 360.], fontsize = 0)
plt.yticks([10., 30., 50., 70., 90.], fontsize = 0)

plt.subplot(2, 3, 4)
plt.title("d)", fontsize = 22)

plt.contourf(lon, lat, -mflow2 * 86400., v, cmap = plt.cm.seismic)
plt.ylim([10., 90.])
levs = [-1000., 1., 1000.]
plt.contour(lon, lat,land2, levs, colors = 'k')
plt.xlabel("Longitude", fontsize = 20)
plt.ylabel("Latitude", fontsize = 20)
plt.xticks([0., 60., 120., 180., 240., 300., 360.], fontsize = 18)
plt.yticks([10., 30., 50., 70., 90.], fontsize = 18)


ax = plt.subplot(2, 3, 5)
plt.title("e)", fontsize = 22)

plt.contourf(lon, lat, -eflow2 * 86400., v, cmap = plt.cm.seismic)
plt.ylim([10., 90.])
plt.contour(lon, lat,land2, levs, colors = 'k')
plt.xlabel("Longitude", fontsize = 20)
#plt.ylabel("Latitude", fontsize = 20)
plt.xticks([0., 60., 120., 180., 240., 300., 360.], fontsize = 18)
plt.yticks([10., 30., 50., 70., 90.], fontsize = 0)

ax = plt.subplot(2, 3, 6)
plt.title("f)", fontsize = 22)

im = plt.contourf(lon, lat, -nlflow * 86400., v, cmap = plt.cm.seismic)
plt.ylim([10., 90.])
plt.contour(lon, lat,land2, levs, colors = 'k')
plt.xlabel("Longitude", fontsize = 20)
#plt.ylabel("Latitude", fontsize = 20)
plt.xticks([0., 60., 120., 180., 240., 300., 360.], fontsize = 18)
plt.yticks([10., 30., 50., 70., 90.], fontsize = 0)


cbar_ax = fig.add_axes([0.92, 0.15, 0.011, 0.74])
cb = fig.colorbar(im, cax=cbar_ax, ticks = [-3., -1.5, 0., 1.5, 3.])
cb.set_label( "[K/day]", fontsize = 22  )
cb.ax.tick_params(labelsize=18) 
plt.savefig("GOM_Inland_temp_advection.png")
plt.savefig("GOM_Inland_temp_advection.pdf")

In [ ]:
#Slanted/Tapered
mflow, eflow, nlflow = calc_temp_adv( "Slanted" )
mflow2, eflow2, nlflow2 = calc_temp_adv( "Tapered" )

In [ ]:
fig = plt.figure(figsize=(22,9))
fig.subplots_adjust(top = 0.9, left = 0.07, bottom = 0.15,  wspace = 0.15, hspace = 0.3)

opt = ["Slanted", "Tapered"]
land = getland(opt[0])
land2 = getland(opt[1])
    
    
v = np.linspace(-3., 3., 20)
plt.subplot(2, 3, 1)
plt.title('a) '+r'$-\overline{[\mathbf{v}]}\cdot\nabla \overline{T^*}$', fontsize = 22)

plt.contourf(lon, lat, -mflow * 86400., v, cmap = plt.cm.seismic)
plt.ylim([10., 90.])
levs = [-1000., 1., 1000.]
plt.contour(lon, lat,land, levs, colors = 'k')
#plt.xlabel("Longitude", fontsize = 20)
plt.ylabel("Latitude", fontsize = 20)
plt.xticks([0., 60., 120., 180., 240., 300., 360.], fontsize = 0)
plt.yticks([10., 30., 50., 70., 90.], fontsize = 18)


ax = plt.subplot(2, 3, 2)
plt.title('b) '+r'$-\overline{\mathbf{v}^*}\cdot\nabla \overline{[T]}$', fontsize = 22)

plt.contourf(lon, lat, -eflow * 86400., v, cmap = plt.cm.seismic)
plt.ylim([10., 90.])
plt.contour(lon, lat,land, levs, colors = 'k')
#plt.xlabel("Longitude", fontsize = 20)
#plt.ylabel("Latitude", fontsize = 20)
plt.xticks([0., 60., 120., 180., 240., 300., 360.], fontsize = 0)
plt.yticks([10., 30., 50., 70., 90.], fontsize = 0)

ax = plt.subplot(2, 3, 3)
plt.title('c) '+r'$-\overline{\mathbf{v}^*}\cdot\nabla \overline{T^*}$', fontsize = 22)

im = plt.contourf(lon, lat, -nlflow * 86400., v, cmap = plt.cm.seismic)
plt.ylim([10., 90.])
plt.contour(lon, lat,land, levs, colors = 'k')
#plt.xlabel("Longitude", fontsize = 20)
#plt.ylabel("Latitude", fontsize = 20)
plt.xticks([0., 60., 120., 180., 240., 300., 360.], fontsize = 0)
plt.yticks([10., 30., 50., 70., 90.], fontsize = 0)

plt.subplot(2, 3, 4)
plt.title("d)", fontsize = 22)

plt.contourf(lon, lat, -mflow2 * 86400., v, cmap = plt.cm.seismic)
plt.ylim([10., 90.])
levs = [-1000., 1., 1000.]
plt.contour(lon, lat,land2, levs, colors = 'k')
plt.xlabel("Longitude", fontsize = 20)
plt.ylabel("Latitude", fontsize = 20)
plt.xticks([0., 60., 120., 180., 240., 300., 360.], fontsize = 18)
plt.yticks([10., 30., 50., 70., 90.], fontsize = 18)


ax = plt.subplot(2, 3, 5)
plt.title("e)", fontsize = 22)

plt.contourf(lon, lat, -eflow2 * 86400., v, cmap = plt.cm.seismic)
plt.ylim([10., 90.])
plt.contour(lon, lat,land2, levs, colors = 'k')
plt.xlabel("Longitude", fontsize = 20)
#plt.ylabel("Latitude", fontsize = 20)
plt.xticks([0., 60., 120., 180., 240., 300., 360.], fontsize = 18)
plt.yticks([10., 30., 50., 70., 90.], fontsize = 0)

ax = plt.subplot(2, 3, 6)
plt.title("f)", fontsize = 22)

im = plt.contourf(lon, lat, -nlflow * 86400., v, cmap = plt.cm.seismic)
plt.ylim([10., 90.])
plt.contour(lon, lat,land2, levs, colors = 'k')
plt.xlabel("Longitude", fontsize = 20)
#plt.ylabel("Latitude", fontsize = 20)
plt.xticks([0., 60., 120., 180., 240., 300., 360.], fontsize = 18)
plt.yticks([10., 30., 50., 70., 90.], fontsize = 0)


cbar_ax = fig.add_axes([0.92, 0.15, 0.011, 0.74])
cb = fig.colorbar(im, cax=cbar_ax, ticks = [-3., -1.5, 0., 1.5, 3.])
cb.set_label( "[K/day]", fontsize = 22  )
cb.ax.tick_params(labelsize=18) 
plt.savefig("Slanted_Tapered_temp_advection.png")
plt.savefig("Slanted_Tapered_temp_advection.pdf")

In [ ]:
#Slanted/Tapered
mflow, eflow, nlflow = calc_temp_adv( "two_continents" )
mflow2, eflow2, nlflow2 = calc_temp_adv( "two_tapered" )

In [ ]:
fig = plt.figure(figsize=(22,9))
fig.subplots_adjust(top = 0.9, left = 0.07, bottom = 0.15,  wspace = 0.15, hspace = 0.3)

opt = ["two_continents", "two_tapered"]
land = getland(opt[0])
land2 = getland(opt[1])
    
    
v = np.linspace(-3., 3., 20)
plt.subplot(2, 3, 1)
plt.title('a) '+r'$-\overline{[\mathbf{v}]}\cdot\nabla \overline{T^*}$', fontsize = 22)

plt.contourf(lon, lat, -mflow * 86400., v, cmap = plt.cm.seismic)
plt.ylim([10., 90.])
levs = [-1000., 1., 1000.]
plt.contour(lon, lat,land, levs, colors = 'k')
#plt.xlabel("Longitude", fontsize = 20)
plt.ylabel("Latitude", fontsize = 20)
plt.xticks([0., 60., 120., 180., 240., 300., 360.], fontsize = 0)
plt.yticks([10., 30., 50., 70., 90.], fontsize = 18)


ax = plt.subplot(2, 3, 2)
plt.title('b) '+r'$-\overline{\mathbf{v}^*}\cdot\nabla \overline{[T]}$', fontsize = 22)

plt.contourf(lon, lat, -eflow * 86400., v, cmap = plt.cm.seismic)
plt.ylim([10., 90.])
plt.contour(lon, lat,land, levs, colors = 'k')
#plt.xlabel("Longitude", fontsize = 20)
#plt.ylabel("Latitude", fontsize = 20)
plt.xticks([0., 60., 120., 180., 240., 300., 360.], fontsize = 0)
plt.yticks([10., 30., 50., 70., 90.], fontsize = 0)

ax = plt.subplot(2, 3, 3)
plt.title('c) '+r'$-\overline{\mathbf{v}^*}\cdot\nabla \overline{T^*}$', fontsize = 22)

im = plt.contourf(lon, lat, -nlflow * 86400., v, cmap = plt.cm.seismic)
plt.ylim([10., 90.])
plt.contour(lon, lat,land, levs, colors = 'k')
#plt.xlabel("Longitude", fontsize = 20)
#plt.ylabel("Latitude", fontsize = 20)
plt.xticks([0., 60., 120., 180., 240., 300., 360.], fontsize = 0)
plt.yticks([10., 30., 50., 70., 90.], fontsize = 0)

plt.subplot(2, 3, 4)
plt.title("d)", fontsize = 22)

plt.contourf(lon, lat, -mflow2 * 86400., v, cmap = plt.cm.seismic)
plt.ylim([10., 90.])
levs = [-1000., 1., 1000.]
plt.contour(lon, lat,land2, levs, colors = 'k')
plt.xlabel("Longitude", fontsize = 20)
plt.ylabel("Latitude", fontsize = 20)
plt.xticks([0., 60., 120., 180., 240., 300., 360.], fontsize = 18)
plt.yticks([10., 30., 50., 70., 90.], fontsize = 18)


ax = plt.subplot(2, 3, 5)
plt.title("e)", fontsize = 22)

plt.contourf(lon, lat, -eflow2 * 86400., v, cmap = plt.cm.seismic)
plt.ylim([10., 90.])
plt.contour(lon, lat,land2, levs, colors = 'k')
plt.xlabel("Longitude", fontsize = 20)
#plt.ylabel("Latitude", fontsize = 20)
plt.xticks([0., 60., 120., 180., 240., 300., 360.], fontsize = 18)
plt.yticks([10., 30., 50., 70., 90.], fontsize = 0)

ax = plt.subplot(2, 3, 6)
plt.title("f)", fontsize = 22)

im = plt.contourf(lon, lat, -nlflow * 86400., v, cmap = plt.cm.seismic)
plt.ylim([10., 90.])
plt.contour(lon, lat,land2, levs, colors = 'k')
plt.xlabel("Longitude", fontsize = 20)
#plt.ylabel("Latitude", fontsize = 20)
plt.xticks([0., 60., 120., 180., 240., 300., 360.], fontsize = 18)
plt.yticks([10., 30., 50., 70., 90.], fontsize = 0)


cbar_ax = fig.add_axes([0.92, 0.15, 0.011, 0.74])
cb = fig.colorbar(im, cax=cbar_ax, ticks = [-3., -1.5, 0., 1.5, 3.])
cb.set_label( "[K/day]", fontsize = 22  )
cb.ax.tick_params(labelsize=18) 
plt.savefig("two_continent_temp_advection.png")
plt.savefig("two_continent_temp_advection.pdf")

Advective terms in variance budget:

In [ ]:
def gradient_3d(mean_temp, lat, lon):
    #gradient of 3D array
    
    d1,d2, d3 = np.shape(mean_temp)
    gradY = np.zeros(((d1, d2, d3)))
    gradX = np.zeros(((d1, d2, d3)))
    cos_mean_temp = np.zeros(((d1, d2, d3)))

    y=len(lat)
    x=len(lon)
    rad_lat = lat * np.pi / 180.
    rad_lon = lon * np.pi / 180.
    
    for i in range( len(lat)):
        cos_mean_temp = mean_temp /np.cos(rad_lat[np.newaxis, :, np.newaxis] )

        if i==0:
            gradY[:, i, :] = ((mean_temp[ :, 1, :] - mean_temp[ :, 0, :]) / (rad_lat[ 1] - rad_lat[ 0])) / 6.371
        elif i== y-1:
            gradY[:, i, :] = ((mean_temp[:, i - 1, :] - mean_temp[:,  i - 2, :]) / (rad_lat[i - 1] - rad_lat[i - 2])) / 6.371
        else:
            gradY[:, i, :] = ((mean_temp[:, i + 1, :] - mean_temp[ :, i - 1, :]) / (rad_lat[i + 1] - rad_lat[i - 1])) / 6.371

    for i in range( len(lon)):
        if i==0:
            gradX[:, :, i] = ((cos_mean_temp[:,  :, 1] - cos_mean_temp[ :, :, 0]) / (rad_lon[ 1] - rad_lon[ 0])) / 6.371
        elif i== x-1:
            gradX[:, :, i] = ((cos_mean_temp[:, :, i-1] - cos_mean_temp[:,  :, i-2])/ (rad_lon[i - 1] - rad_lon[i - 2])) / 6.371
        else:
            gradX[:, :, i] = ((cos_mean_temp[:, :, i+1] - cos_mean_temp[:, :, i-1])/ (rad_lon[i + 1] - rad_lon[i - 1])) / 6.371
    return gradY, gradX

def get_variance_budget_advective(opt):
    data=getdata(opt)
    
    temp_850 = data['temp'].interp(pfull=850)
    vcomp_850 = data['vcomp'].interp(pfull=850)
    ucomp_850 = data['ucomp'].interp(pfull=850)
    
    lons = np.zeros(256)
    lons[:] = data['lon'][:]
    lats = np.zeros(128)
    lats[:] = data['lat'][:]
    
    term1 = np.zeros((( 15, 128, 256)))
    term2 = np.zeros((( 15, 128, 256)))
    term3= np.zeros((( 15, 128, 256)))

    a = 675
    for i in range( 15 ):
        print("Doing:", i)
        wint_temp = temp_850[a: a + 90, :, :]
        wint_ucomp = ucomp_850[a: a + 90, :, :]
        wint_vcomp = vcomp_850[a: a + 90, :, :]
        
        Um = np.zeros( (128, 256))
        Vm = np.zeros( (128, 256))
        Tm = np.zeros( (128, 256))
        
        Um[:, :] = np.mean(wint_ucomp, axis = 0)
        Vm[:, :] = np.mean(wint_vcomp, axis = 0)
        Tm[:, :] = np.mean(wint_temp, axis = 0)

        U_e = wint_ucomp - Um[np.newaxis, :, :]
        V_e = wint_vcomp - Vm[np.newaxis, :, :]
        T_e = wint_temp - Tm[np.newaxis, :, :]

        grad_Tey, grad_Tex = gradient_3d( T_e ** 2 / 2., lats, lons)
        grad_Tmy, grad_Tmx = gradient( Tm, lats, lons)

        term1[i] = np.mean(Um[np.newaxis, :, :] * grad_Tex + Vm[np.newaxis, :, :] * grad_Tey, axis = 0)
        term2[i] = np.mean(U_e * T_e, axis = 0) * grad_Tmx + np.mean(V_e * T_e, axis = 0) * grad_Tmy
        term3[i] = np.mean(U_e * grad_Tex + V_e * grad_Tey, axis = 0)         
        a += 360
        
    return np.mean(term1, axis = 0), np.mean(term2, axis = 0), np.mean(term3, axis = 0)

In [ ]:
term1, term2, term3 = get_variance_budget_advective("control")

#convert to K^2 /day (and not that gradients were taken in 10^3km, so need to convert to m)
term1 /= 10. ** 6 / 86400.
term2 /= 10. ** 6 / 86400.
term3 /= 10. ** 6 / 86400. 

land = getland("control")

fig = plt.figure(figsize=(22,5))
fig.subplots_adjust(top = 0.9, left = 0.07, bottom = 0.15,  wspace = 0.2, hspace = 0.45)

v = np.linspace(-46., 46, 20)
plt.subplot(1, 3, 1)
plt.title("a) $-\\bar{\\mathbf{v}}\\cdot\\nabla(\overline{T'^2}/2)$", fontsize = 20)

plt.contourf(lons, lats, -term1, v, cmap = plt.cm.seismic)
plt.ylim([10., 90.])
levs = [-1000., 1., 1000.]
plt.contour(lons, lats,land, levs, colors = 'k')
plt.xlabel("Longitude", fontsize = 20)
plt.ylabel("Latitude", fontsize = 20)
plt.xticks([0., 60., 120., 180., 240., 300., 360.], fontsize = 18)
plt.yticks([10., 30., 50., 70., 90.], fontsize = 18)


ax = plt.subplot(1, 3, 2)
plt.title("b) $-\\overline{\\mathbf{v}'T'}\\cdot\\nabla \\bar{T}$", fontsize = 20)

plt.contourf(lons, lats, -term2, v, cmap = plt.cm.seismic)
plt.ylim([10., 90.])
plt.contour(lons, lats,land, levs, colors = 'k')
plt.xlabel("Longitude", fontsize = 20)
#plt.ylabel("Latitude", fontsize = 20)
plt.xticks([0., 60., 120., 180., 240., 300., 360.], fontsize = 18)
plt.yticks(fontsize = 0)

ax = plt.subplot(1, 3, 3)
plt.title("c) $-\\overline{\\mathbf{v}'\\cdot\\nabla(T'^2/2)}$", fontsize = 20)

im = plt.contourf(lons, lats, -term3, v, cmap = plt.cm.seismic)
plt.ylim([10., 90.])
plt.contour(lons, lats,land, levs, colors = 'k')
plt.xlabel("Longitude", fontsize = 20)
#plt.ylabel("Latitude", fontsize = 20)
plt.xticks([0., 60., 120., 180., 240., 300., 360.], fontsize = 18)
plt.yticks(fontsize = 0)


cbar_ax = fig.add_axes([0.92, 0.15, 0.011, 0.74])
cb = fig.colorbar(im, cax=cbar_ax, ticks = [-45., -30., -15., 0., 15., 30., 45.])
cb.set_label( "[K$^2$ / day]", fontsize = 20  )
cb.ax.tick_params(labelsize=16) 

plt.savefig("Temperature_variance_budget_control.png")
plt.savefig("Temperature_variance_budget_control.pdf")

Extra plot to illustrate connection to EKE

In [ ]:
var_temp, Zsub_temp, gradY, gradX, Zsub_press, u_850, v_850, Zsub_ucomp, Zsub_vcomp, lon, lat  = get_data( "control")
var_temp2, Zsub_temp2, gradY2, gradX2, Zsub_press2, u_8502, v_8502, Zsub_ucomp2, Zsub_vcomp2, lon, lat  = get_data( "Inland_Sea")


In [ ]:
fig = plt.figure(figsize=(14,6))
fig.subplots_adjust(right = 0.97, top = 0.97, left = 0.1, bottom = 0.2,  wspace = 0.3, hspace = 0.45)

opt = ["control", "Inland_Sea"]
land = getland(opt[0])
land2 = getland(opt[1])
   
    
ax = plt.subplot(2, 2, 1)
plt.title("a) Control", loc = "left", fontsize = 15)

v = np.arange(10., 99., 9.)
plt.contour( lons, lats, temp_var, v, colors = 'r', linewidths = 0.5)
v2 = np.arange(20., 180., 10.)
plt.contourf( lons, lats, eke, v, cmap = plt.cm.Blues)
plt.contour(lons, lats, land, levels=1,colors='k', linewidths = 0.5, alpha = 0.5)

plt.ylim([10., 80.])
plt.ylabel("Latitude", fontsize = 14)
plt.xticks(fontsize = 0)
plt.yticks([10., 30., 50., 70.], fontsize = 12)

ax = plt.subplot(2, 2, 2)
plt.title("b) Inland Sea", loc = "left", fontsize = 15)

plt.contour( lons, lats, temp_var2, v, colors = 'r', linewidths = 0.5)
C1 = plt.contourf( lons, lats, eke2, v, cmap = plt.cm.Blues)
plt.contour(lons, lats, land2, levels=1,colors='k', linewidths = 0.5, alpha = 0.5)

plt.ylim([10., 80.])
plt.xticks(fontsize = 0)
plt.yticks([10., 30., 50., 70.], fontsize = 0)

ax = plt.subplot(2, 2, 3)
plt.title("c)", loc = "left", fontsize = 15)

v = np.arange(10., 99., 9.)
plt.contour( lons, lats, temp_var, v, colors = 'r', linewidths = 0.5)

levs = np.arange(0.1, 1.7, 0.1)
plt.contourf( lons, lats, (gradY ** 2 + gradX ** 2) / 100., levs, cmap = plt.cm.Blues)
plt.contour(lons, lats, land, levels=1,colors='k', linewidths = 0.5, alpha = 0.5)

plt.ylim([10., 80.])
plt.xlabel("Longitude", fontsize = 14)
plt.ylabel("Latitude", fontsize = 14)
plt.xticks(fontsize = 12)
plt.yticks([10., 30., 50., 70.], fontsize = 12)

ax = plt.subplot(2, 2, 4)
plt.title("d)", loc = "left", fontsize = 15)

plt.contour( lons, lats, temp_var2, v, colors = 'r', linewidths = 0.5)
C2 = plt.contourf( lons, lats, (gradY2 ** 2 + gradX2 ** 2) / 100., levs, cmap = plt.cm.Blues)
plt.contour(lons, lats, land, levels=1,colors='k', linewidths = 0.5, alpha = 0.5)


plt.ylim([10., 80.])
plt.xlabel("Longitude", fontsize = 14)
plt.xticks(fontsize = 12)
plt.yticks([10., 30., 50., 70.], fontsize = 0)

fig.tight_layout()

fig.subplots_adjust(right = 0.86)
cbar_ax = fig.add_axes([0.88, 0.57, 0.01, 0.365])
cb = fig.colorbar(C1, cax=cbar_ax)
cb.set_label( 'EKE [m$^{2}$s$^{-2}$]', fontsize = 14  )
cb.ax.tick_params(labelsize=12)


cbar_ax2 = fig.add_axes([0.88, 0.11, 0.01, 0.365])
cb2 = fig.colorbar(C2, cax=cbar_ax2)
cb2.set_label( r'$\left(\frac{\partial \overline{T}}{\partial x}\right)^2 + \left(\frac{\partial \overline{T}}{\partial y}\right)^2$' + " [(K/100km)$^2$]", fontsize = 14  )
cb2.ax.tick_params(labelsize=12)


plt.savefig("Control_GOM_eke_comp.png")
plt.savefig("Control_GOM_eke_comp.pdf")